In [ ]:
import numpy as np
# Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:

# Load the dataset
df = pd.read_excel('DQN1 Dataset.xlsx')
# Take the first look at the dataset
df.info()

In [ ]:

# Prepare data for model
X = df.drop('healthRiskScore', axis=1)
y = df['healthRiskScore']
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=7)  # 30% test, 70% training

In [ ]:

# Create the first model
lr_model = LinearRegression()
# Fit the model
lr_model.fit(X_train, y_train)
# Predict using the model
y_pre_lr = lr_model.predict(X_test)


In [ ]:
# B1. First Optimization technique using 
# Stochastic Gradient Descent with StandardScaler for supporting
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('linear', SGDRegressor(loss='squared_error', learning_rate='adaptive'))
])
# Fit the pipeline to the data
pipe.fit(X_train, y_train)
# Predict using the model
y_pre_SGD = pipe.predict(X_test)


In [ ]:
# B1. Second Optimization technique using 
# Huber Loss with StandardScaler for supporting
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
# Create pipeline to house scaler and estimator
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('linear', HuberRegressor(epsilon=1.35, max_iter= 10000))
])
# Fit the pipeline to the data
pipe.fit(X_train, y_train)
# Predict using the model
y_pre_HR = pipe.predict(X_test)
